In [ ]:
from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, QuantumRegister
from qiskit.qasm import pi
from qiskit.tools.visualization import plot_histogram, circuit_drawer
from qiskit import execute, Aer, BasicAer
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from resizeimage import resizeimage
import warnings
warnings.filterwarnings('ignore')
from qiskit.aqua.circuits.fourier_transform_circuits import FourierTransformCircuits
from scipy import misc
from matplotlib.image import imread

In [ ]:
import frqi                                                                     
import quantum_edge_detection as qed

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q-university", group="columbia-univers", project="comse-6998-2020")

In [ ]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cu1(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [ ]:
def swap_x_y(circ, img):
    lencirc = int(len(img)/2)
    start = 1
    end = start + lencirc
    for x in range(1, end):
        circ.swap(img[x], img[x+lencirc])
    return circ

In [ ]:
def flip(circ, img, start):
    end = start + int(len(img)/2)
    for i in range(start, end):
        circ.x(img[i])
    return circ

def flip_x(circ, img):
    start = 1 + int(len(img)/2)
    return flip(circ, img, start)

def flip_y(circ, img):
    start = 1
    return flip(circ, img, start)

In [ ]:
def r90(circ, img):
    return swap_x_y(flip_y(circ, img), img)

def r180(circ, img):
    return flip_x(flip_y(circ, img), img)

def r270(circ, img):
    return swap_x_y(flip_x(circ, img), img)

In [ ]:
def c3x(circ, img, ans, dest, otr):
    first = otr[1]
    second = otr[2]
    third = otr[3]
    circ.ccx(img[second], img[first], img[ans])
    circ.ccx(img[ans], img[third], img[dest])
    circ.ccx(img[second], img[first], img[ans])
    circ.ccx(img[ans], img[third], img[dest])
    return circ
def c5x():
    return None
def c7x():
    return None
def c9x():
    return None
def c11x():
    return None


In [ ]:
def color_flip(circ, img):
    circ.x(img[0])
    return circ
def rotation_color_angle(circ, img, angle):
    imglen = len(img)
    circ.mcry(angle, [img[j] for j in [4,5,6,7,8,9,10]], img[0], None, mode='noancilla')
    circ.mcry(angle/2, [img[j] for j in [1,2,3,6,7,8,9,10]], img[0], None, mode='noancilla')
    return circ
    #circ.mcry

def trial_color_angle(circ, img, angle):
    imglen = len(img)
    circ.mcry(angle, [img[j] for j in [1,2,3,6,7,8]], img[0], None, mode='noancilla')
    #circ.mcry(angle/2, [img[j] for j in [1,2,3,6,7,8,9,10]], img[0], None, mode='noancilla')
    return circ

In [ ]:
anc = QuantumRegister(1, "anc")
img = QuantumRegister(11, "img")
anc2 = QuantumRegister(1, "anc2")

anc22 = QuantumRegister(1, "anc22")
img2 = QuantumRegister(11, "img2")
anc222 = QuantumRegister(1, "anc222")

anc_col = QuantumRegister(1, "anc_col")
img_col = QuantumRegister(14, "img_col")
anc2_col = QuantumRegister(4, "anc2_col")

c = ClassicalRegister(12)
c2 = ClassicalRegister(12)
c_col = ClassicalRegister(15)

qc = QuantumCircuit(anc, img, anc2,c)
qc2 = QuantumCircuit(anc22, img2, anc222,c2)
qc_col = QuantumCircuit(anc_col, img_col, anc2_col,c_col)

imageNames = ["images.jpeg","download.jpeg","Pic.jpg","Pic2.jpg","Original 640x426/Tokyo - ISS030-E-82770.JPG", "Original 640x426/Osaka - ISS030-E-184140.JPG", "Original 640x426/Nagoya - ISS038-E-11205.JPG"]
imageNum1 = 0
imageNum2 = 0

image1 = Image.open(imageNames[imageNum1]).convert('LA')
image2 = Image.open(imageNames[imageNum2])
image2a = Image.open(imageNames[imageNum2])

def image_normalization(image):
    image = resizeimage.resize_cover(image, [32, 32])
    w, h = 32, 32
    image = np.array([[image.getpixel((x,y))[0] for x in range(w)] for y in range(h)])

    # 2-dimentional data convert to 1-dimentional array
    image = image.flatten()
    # change type
    image = image.astype('float64')
    # Normalization(0~pi/2)
    image /= 255.0
    generated_image = np.arcsin(image)

    return generated_image

def image_normalization_col(image):
    image = image.resize((32,32),resample=Image.NEAREST)
    px = image.load()
    w, h = 32, 32
    imagea = np.zeros((32,32)) + 1
    imager = np.array([[image.getpixel((x,y))[0] for x in range(w)] for y in range(h)])
    imageg = np.array([[image.getpixel((x,y))[1] for x in range(w)] for y in range(h)])
    imageb = np.array([[image.getpixel((x,y))[2] for x in range(w)] for y in range(h)])

    # 2-dimentional data convert to 1-dimentional array
    imagea = imagea.flatten()
    imager = imager.flatten()
    imageg = imageg.flatten()
    imageb = imageb.flatten()
    
    # change type
    
    imagea = imagea.astype('float64')
    imager = imager.astype('float64')
    imageg = imageg.astype('float64')
    imageb = imageb.astype('float64')
    # Normalization(0~pi/2)
    imager /= 255.0
    imageg /= 255.0
    imageb /= 255.0
    
    generated_imager = np.arcsin(imagea)
    generated_imager = np.arcsin(imager)
    generated_imageg = np.arcsin(imageg)
    generated_imageb = np.arcsin(imageb)

    return np.concatenate((imagea, imager, imageg, imageb), axis=None)

image1 = image_normalization(image1)
image2 = image_normalization(image2)
image1c = image_normalization_col(image2a)

In [ ]:
# apply hadamard gates
for i in range(1, len(img)):
    qc.h(img[i])
for i in range(1, len(img2)):
    qc2.h(img2[i])

for i in range(4, len(img_col)):
    qc_col.h(img_col[i])
    
    
# encode ref image
for i in range(len(image1)):
        if image1[i] != 0:
                frqi.c10ry(qc, 2 * image1[i], format(i, '010b'), img[0], anc2[0], [img[j] for j in range(1,len(img))])

for i in range(len(image2)):
        if image2[i] != 0:
                frqi.c10mary(qc2, 2 * image2[i], format(i, '010b'), img2[0], anc222[0], [img2[j] for j in range(1,len(img2))])
offseta = 0
offsetr = 32 * 32
offsetg = offsetr * 2
offsetb = offsetr * 3

for i in range(len(image1)):
        if image1c[i+offseta] != 0:
                frqi.c10mary(qc_col, 2 * image1c[i+offseta], format(i, '010b'), img_col[0], anc2_col[0], [img_col[j] for j in range(4,len(img_col))])
        if image1c[i+offsetr] != 0:
                frqi.c10mary(qc_col, 2 * image1c[i+offsetr], format(i, '010b'), img_col[1], anc2_col[1], [img_col[j] for j in range(4,len(img_col))])
        if image1c[i+offsetg] != 0:
                frqi.c10mary(qc_col, 2 * image1c[i+offsetg], format(i, '010b'), img_col[2], anc2_col[2], [img_col[j] for j in range(4,len(img_col))])
        if image1c[i+offsetb] != 0:
                frqi.c10mary(qc_col, 2 * image1c[i+offsetb], format(i, '010b'), img_col[3], anc2_col[3], [img_col[j] for j in range(4,len(img_col))])
                
#qed.quantum_edge_detection(qc)
#FourierTransformCircuits.construct_circuit(qc, qc.qubits)
#print(len(qc.qubits))
#qft(qc, len(qc.qubits))
#epsilon = -1
#qubits = qc.qubits
#n_qubits = qc.n_qubits
#for i in range(n_qubits):
#    qc.u1(float(2 * pi * epsilon)/2**(i + 1), qubits[n_qubits - i - 1])


qc.measure(anc, c[0])
qc.measure(img, c[1:12])

#swap_x_y(qc2, img2)
#flip_y(qc2, img2)
#r270(qc2, img2)
#color_flip(qc2, img2)
#rotation_color_angle(qc2, img2, (2*np.pi/3))
#trial_color_angle(qc2, img2, (np.pi/12.5))

qc2.measure(anc22,c2[0])
qc2.measure(img2, c2[1:12])

qc_col.measure(anc_col, c_col[0])
qc_col.measure(img_col, c_col[1:15])

numOfShots = 1024*1024*8
#result = execute(qc, provider.get_backend('ibmq_qasm_simulator'), shots=numOfShots, backend_options={"fusion_enable":True}).result()
backend = Aer.get_backend('qasm_simulator') 
#result = execute(qc, backend, shots=numOfShots, backend_options={"fusion_enable":True}).result()
#result2 = execute(qc2, backend, shots=numOfShots, backend_options={"fusion_enable":True}).result()
result_col = execute(qc_col, backend, shots=numOfShots, backend_options={"fusion_enable":True}).result()

In [ ]:
#print(result.get_counts(qc))
#print(int(len(img)/2))

In [ ]:
# generated image
genimg = np.array([])
genimg2 = np.array([])
genimg_col = np.array([])
#### decode
'''
for i in range(len(image1)):
        try:
                genimg = np.append(genimg,[np.sqrt(result.get_counts(qc)[format(i, '010b')+'10']/numOfShots)])
        except KeyError:
                genimg = np.append(genimg,[0.0])
for i in range(len(image2)):
        try:
                genimg2 = np.append(genimg2,[np.sqrt(result2.get_counts(qc2)[format(i, '010b')+'10']/numOfShots)])
        except KeyError:
                genimg2 = np.append(genimg2,[0.0])
'''

for i in range(len(image1)):
        try:
                genimg_col = np.append(genimg_col,[np.sqrt(result_col.get_counts(qc_col)[format(i, '010b')+'00010']/numOfShots)])
        except KeyError:
                genimg_col = np.append(genimg_col,[0.0])


for i in range(len(image1)):
        try:
                genimg_col = np.append(genimg_col,[np.sqrt(result_col.get_counts(qc_col)[format(i, '010b')+'00100']/numOfShots)])
        except KeyError:
                genimg_col = np.append(genimg_col,[0.0])


for i in range(len(image1)):
        try:
                genimg_col = np.append(genimg_col,[np.sqrt(result_col.get_counts(qc_col)[format(i, '010b')+'01000']/numOfShots)])
        except KeyError:
                genimg_col = np.append(genimg_col,[0.0])


for i in range(len(image1)):
        try:
                genimg_col = np.append(genimg_col,[np.sqrt(result_col.get_counts(qc_col)[format(i, '010b')+'10000']/numOfShots)])
        except KeyError:
                genimg_col = np.append(genimg_col,[0.0])
                

#genimg = np.log(np.abs(genimg))
# inverse nomalization
#genimg *= 32.0 * 255.0
#genimg2 *= 32.0 * 255.0
genimg_col *= 32.0 * 255.0

# convert type
#genimg = genimg.astype('int')
#genimg2 = genimg2.astype('int')
genimg_col = genimg_col.astype('int')

# back to 2-dimentional data
#genimg = genimg.reshape((32,32))
#genimg2 = genimg2.reshape((32,32))
genimg_col = genimg_col.reshape((32,32,4))

#plt.imshow(genimg, cmap='gray', vmin=0, vmax=255)
#plt.savefig('gen_'+str(imageNum1)+'.png')
#plt.show()


In [ ]:
genimg_rgb = genimg_col[:,:,1:4]

In [ ]:
plt.imshow(genimg_rgb, cmap='gray')

In [ ]:
plt.imshow(genimg2, cmap='gray', vmin=0, vmax=255)
#plt.savefig('gen_'+str(imageNum1)+'.png')
plt.show()

In [ ]:
from numpy.fft import fft2, fftshift, ifftshift, ifft2
fft_new = fftshift(fft2(image1.reshape(32,32)))
img_ff = np.log(np.abs(fft_new))
plt.imshow(img_ff, cmap='gray')
#plt.savefig('gen_'+str(imageNum1)+'.png')
plt.show()

In [ ]:
from IPython.display import Image as show_img
show_img(filename='Pic.jpg')
#show_img(filename='Pic2.jpg')

In [ ]:
from scipy.signal import convolve2d
def gray2rgb(image):
    return np.repeat(np.expand_dims(image, 2), 3, axis=2)

def load_image(filename):
    img = np.asarray(Image.open(filename))
    img = img.astype("float32") / 255.
    return img

def show_image(img):
    if len(img.shape) == 2:
        img = gray2rgb(img)
    plt.imshow(img, interpolation='nearest')

In [ ]:
im = load_image(imageNames[imageNum1])
im = im.mean(axis=2) # convert to grayscale
show_image(im/2)

In [ ]:
#delta_filt = np.array([1,-1,-1,1]).reshape(2,2)
delta_filt = np.array([1,-1]).reshape(1,2)
plt.imshow(convolve2d(im, delta_filt), cmap='gray')

In [ ]:

image = imread('images.jpeg')
print(image.shape)
plt.imshow(image, cmap='gray')

In [ ]:
from qiskit import IBMQ
IBMQ.save_account('d6b8341cf93840fcc1102abcda046ffed464d0d10c968ea9ebcedcb76044fe3732c7bed54e2b697fd4c8f8ed42676775f341d05509d59c157a7ebd410630e6c2',overwrite=True)

In [ ]:
image2 = imread(imageNames[imageNum2])
image2 = Image.open(imageNames[imageNum2])
img = image2.resize((32,32), resample=Image.NEAREST)
im = image2.load()
plt.imshow(image2, cmap='gray')
print(im[2,2])
print(img.size)
#image2 = resizeimage.resize_cover(image2, [32, 32,3])